In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
import pandas as pd

# Read the general dataset
df = pd.read_csv("/content/drive/MyDrive/dataset.csv")
print("Preview of general dataset:")
print(df.head())

# Read the Russia-Ukraine parsed dataset
df_parsed = pd.read_csv("/content/drive/MyDrive/parsed_articles.csv")

# Convert labels from 'real'/'fake' to 0/1
df_parsed['label'] = df_parsed['label'].astype(str).str.strip().replace({'real': 0, 'fake': 1})
df_parsed['label'] = df_parsed['label'].astype(int)

# Read the general dataset again for sampling
df_general = pd.read_csv("/content/drive/MyDrive/dataset.csv")

# Clean and normalize the labels in general dataset
df_general['label'] = df_general['label'].astype(str).str.strip().replace({'real': 0, 'fake': 1})
df_general['label'] = pd.to_numeric(df_general['label'], errors='coerce')
df_general = df_general[df_general['label'].isin([0, 1])]  # Keep only 0 or 1 labels
df_general = df_general[['text', 'label']].dropna()        # Keep only text and label columns
df_general['label'] = df_general['label'].astype(int)

# Count how many real and fake articles already exist
real_parsed = df_parsed[df_parsed['label'] == 0]
fake_parsed = df_parsed[df_parsed['label'] == 1]

# We want 500 real and 500 fake articles total
real_needed = max(0, 500 - len(real_parsed))
fake_needed = max(0, 500 - len(fake_parsed))

# Get extra real/fake articles from the general dataset if needed
real_pool = df_general[df_general['label'] == 0]
fake_pool = df_general[df_general['label'] == 1]

# Sample only if needed and enough samples exist
real_extra = real_pool.sample(n=real_needed, random_state=12) if real_needed > 0 and len(real_pool) >= real_needed else pd.DataFrame(columns=df_general.columns)
fake_extra = fake_pool.sample(n=fake_needed, random_state=12) if fake_needed > 0 and len(fake_pool) >= fake_needed else pd.DataFrame(columns=df_general.columns)

# Combine the original and extra data
df_final = pd.concat([real_parsed, fake_parsed, real_extra, fake_extra], ignore_index=True)

# Shuffle the final dataset to randomize the order
df_final = df_final.sample(frac=1, random_state=12).reset_index(drop=True)

# Save the final dataset to a CSV file
df_final.to_csv("/content/drive/MyDrive/final_russia_ukraine_dataset.csv", index=False)

# Show summary
print("Final dataset shape:", df_final.shape)
print("Label distribution:")
print(df_final['label'].value_counts())
print(df_final)

Preview of general dataset:
   id                                              title  author  \
0   0  Ξεκινά 1η Ιουνίου εμβολιασμός για προστασία απ...     NaN   
1   1  Προμηθέας: Στην καλύτερη πεντάδα του Adidas Ne...     NaN   
2   2  Πήρε «φωτιά» το 1110: Εκατοντάδες καταγγελίες ...     NaN   
3   3  Από ισχυρό μετατραυματικό στρες υποφέρει ο Θάν...     NaN   
4   4  Βρετανία: Φτάνει στο αποκορύφωμα του ο κορωνοϊ...     NaN   

                                                text label  
0  Εδωσε το πράσινο φως ο ΕΟΦ – Δείτε αναλυτικά π...     1  
1  Η παρουσία του Προμηθέα Πάτρας στο Adidas Next...     0  
2  Εντατικούς ελέγχους στην αγορά για την πάταξη ...     0  
3  Σε κάκιστη ψυχολογική κατάσταση βρίσκεται τις ...     1  
4  Σύμφωνα με τις δηλώσεις του υπουργού Υγείας, τ...     0  


/tmp/ipython-input-47482244.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_parsed['label'] = df_parsed['label'].astype(str).str.strip().replace({'real': 0, 'fake': 1})


Final dataset shape: (1000, 2)
Label distribution:
label
1    500
0    500
Name: count, dtype: int64
                                                  text  label
0    Ορατό από το διάστημα το πυρηνικό «μανιτάρι» μ...      1
1    Η ΡΩΣΙΑ ΔΕΝ ΠΡΟΚΕΙΤΑΙ ΝΑ ΕΙΣΒΑΛΕΙ ΣΤΗΝ ΟΥΚΡΑΝΙ...      1
2    Φιάσκο οι διαπραγματεύσεις στην Κωνσταντινούπο...      1
3    Η Ρωσία εξαπέλυσε τη μεγαλύτερη αεροπορική επί...      0
4    Ουκρανικό: Σε ειρηνευτική συνάντηση πάνε Ρωσία...      0
..                                                 ...    ...
995  Γιατί ο Μπάιντεν επιτρέπει τώρα στην Ουκρανία ...      0
996  Ουκρανία: Μπαράζ ρωσικών επιθέσεων κατά τη διά...      0
997  Ουκρανία: Δορυφορικές εικόνες αποκαλύπτουν το ...      0
998  ΒΙΝΤΕΟ ΦΡΙΚΗ! Εγκλήματα του ΟυκροΡάιχ που ξεπέ...      1
999  Ξεκίνησε το αντάρτικο στην Ουκρανία - Αντιστασ...      1

[1000 rows x 2 columns]


In [36]:
import pandas as pd

# Load the Russia–Ukraine dataset (balanced: 500 real +500 fake)
df_russia = pd.read_csv("/content/drive/MyDrive/final_russia_ukraine_dataset.csv")

# Load the general dataset from your file
df_general = pd.read_csv("/content/drive/MyDrive/dataset.csv")

# Make sure labels are numbers (0 = real, 1 = fake)
df_general['label'] = df_general['label'].astype(str).str.strip()
df_general['label'] = df_general['label'].replace({'real': 0, 'fake': 1})
df_general['label'] = pd.to_numeric(df_general['label'], errors='coerce')
df_general = df_general[df_general['label'].isin([0, 1])]  # Keep only valid labels

# Select 500 real and 500 fake articles from the github dataset
real_general = df_general[df_general['label'] == 0].sample(n=500, random_state=12)
fake_general = df_general[df_general['label'] == 1].sample(n=500, random_state=12)

# Combine the two groups into one dataframe
df_general_balanced = pd.concat([real_general, fake_general], ignore_index=True)

# Combine Russia–Ukraine dataset with the github dataset
df_combined = pd.concat([df_russia, df_general_balanced], ignore_index=True)

# Keep only 'text' and 'label' columns
df_combined = df_combined[['text', 'label']]

# Shuffle the combined dataset
df_combined = df_combined.sample(frac=1, random_state=12).reset_index(drop=True)

# Save the final combined dataset to a CSV file
df_combined.to_csv("/content/drive/MyDrive/combined_final_dataset.csv", index=False)

print("Final dataset size:", df_combined.shape)
print(df_combined['label'].value_counts())
print(df_combined)

Final dataset size: (2000, 2)
label
0.0    1000
1.0    1000
Name: count, dtype: int64
                                                   text  label
0     Ζελένσκι: Η ένταξη στο ΝΑΤΟ και οι ειρηνευτικέ...    0.0
1     Καμία ξένη δύναμη δεν βρίσκεται σε ελληνικό έδ...    0.0
2     Έτοιμοι να διαφύγουν στο εξωτερικό Ζελένσκι κα...    1.0
3     …μας το έστειλε ένας φίλος… Ένας άνθρωπος που ...    1.0
4     Οδοιπορικό σε ένα απομακρυσμένο και για πολλού...    1.0
...                                                 ...    ...
1995  Με επιστολή του υφυπουργός αθλητισμού, Λευτέρη...    0.0
1996  O ΠΑΟΚ πήρε πολύτιμη όσο και δύσκολη νίκη στο ...    0.0
1997  Δείτε όλες τις τελευταίες Ειδήσεις από την Ελλ...    0.0
1998  Εντονες αντιδράσεις έχει προκαλέσει το γεγονός...    1.0
1999  Την αξιοποίηση του δημοφιλή τραγουδιστή Γιάννη...    1.0

[2000 rows x 2 columns]
